# A lesson on dealing with missing values
Most machine learning libraries can't work with datasets that have nulls in them.
There are different approaches for dealing with these missing values.

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

from intermediate_ml import score_dataset

## First, as usual, the setup

In [12]:
# Read the data
X_full = pd.read_csv('/home/rene/coding/kaggle/intermediate-ml/data/train.csv', index_col='Id')
X_test_full = pd.read_csv('/home/rene/coding/kaggle/intermediate-ml/data/test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# To keep things simple, we'll use only numerical predictors
X = X_full.select_dtypes(exclude=['object'])
X_test = X_test_full.select_dtypes(exclude=['object'])

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [13]:
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,...,774,0,108,0,0,260,0,0,7,2007
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,...,308,0,0,0,0,0,0,0,8,2009
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,...,432,0,0,44,0,0,0,0,8,2009
818,20,NaN,13265,8,5,2002,2002,148.0,1218,0,...,857,150,59,0,0,0,0,0,7,2008
303,20,118.0,13704,7,5,2001,2002,150.0,0,0,...,843,468,81,0,0,0,0,0,1,2006


## Then an investigation into columns with missing values

In [14]:
# Shape of training data (num_rows, num_columns)
print(f"Shape of the training data: {X_train.shape}")
# Number of missing values in each column of training data
missing_val_count_by_column = X_train.isnull().sum()
missing_val_count_by_column = missing_val_count_by_column[missing_val_count_by_column > 0]
print("\nMissing values per column:")
print(missing_val_count_by_column)
# Total number of missing records
print(f"\nTotal number of missing values: {missing_val_count_by_column.sum()}")

Shape of the training data: (1168, 36)

Missing values per column:
LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64

Total number of missing values: 276


## Test runs of three strategies for dealing with missing values

### Approach 1: Just *drop* any column that has missing values
When columns only have a small percentage of missing values, this approach is suboptimal. You will end up discarding a lot of useful information.

In [15]:
# Get names of columns with missing values
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]

# Drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print("MAE from Approach 1 (Drop columns with missing values):")
print(f"{score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid):13.2f}")

MAE from Approach 1 (Drop columns with missing values):
     17837.83


This is the benchmark to run the other methods against.

### Approach 2: *Impute* missing values
This is often a good strategy. By filling missing values with the *mean* of the column, models with slightly sparse data tend to perform better.
There are also more sophisticated imputation strategies, but they don't always lead to better predictions.

In [16]:
# Imputation
mean_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(mean_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(mean_imputer.transform(X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE from Approach 2 (Imputation):")
print(f"{score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid):13.2f}")

MAE from Approach 2 (Imputation):
     18062.89


In this particular dataset imputation even leads to a worse performing model!  
There is no one-size-fits-all approach. In this case, missing values in the `GarageYrBlt` column encode actual missing garages - probably a large influence when it comes to house prices. So replacing blanks with some number throws off the model more than it helps.   

### Approach 3: *Impute* missing values **and** add a feature denoting that the value was changed.
This might make the model more accurate in some situations than just imputing values alone.

In [17]:
# Make copy to avoid changing original data (when imputing)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Make new columns indicating what will be imputed
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# Imputation
mean_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(mean_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(mean_imputer.transform(X_valid_plus))

# Imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE from Approach 3 (An Extension to Imputation):")
print(f"{score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid):13.2f}")

MAE from Approach 3 (An Extension to Imputation):
     18148.42


This time it made things worse unfortunately...

## Other imputation strategies
The `SimpleImputer` comes with four strategies: `mean` (the default), `most_frequent`, `constant` and `median`.    
We have already tested the `mean` strategy. Now we can try the rest and compare the results.

### Most frequent

### 

In [18]:
# Imputation
most_frequent_imputer = SimpleImputer(strategy='most_frequent')
most_frequent_X_train = pd.DataFrame(most_frequent_imputer.fit_transform(X_train))
most_frequent_X_valid = pd.DataFrame(most_frequent_imputer.transform(X_valid))

# Imputation removed column names; put them back
most_frequent_X_train.columns = X_train.columns
most_frequent_X_valid.columns = X_valid.columns

print("MAE from Approach 4 (Imputation with most_frequent):")
print(f"{score_dataset(most_frequent_X_train, most_frequent_X_valid, y_train, y_valid):13.2f}")

MAE from Approach 4 (Imputation with most_frequent):
     17956.07


### Constant
Here we will choose `0` as the fill value, which is also the default.

In [19]:
# Imputation
constant_imputer = SimpleImputer(strategy='constant', fill_value=0)
constant_X_train = pd.DataFrame(constant_imputer.fit_transform(X_train))
constant_X_valid = pd.DataFrame(constant_imputer.transform(X_valid))

# Imputation removed column names; put them back
constant_X_train.columns = X_train.columns
constant_X_valid.columns = X_valid.columns

print("MAE from Approach 5 (Imputation with constant):")
print(f"{score_dataset(constant_X_train, constant_X_valid, y_train, y_valid):13.2f}")

MAE from Approach 5 (Imputation with constant):
     18017.67


### Median

In [20]:
# Imputation
median_imputer = SimpleImputer(strategy='median')
median_X_train = pd.DataFrame(median_imputer.fit_transform(X_train))
median_X_valid = pd.DataFrame(median_imputer.transform(X_valid))

# Imputation removed column names; put them back
median_X_train.columns = X_train.columns
median_X_valid.columns = X_valid.columns

print("MAE from Approach 6 (Imputation with median):")
print(f"{score_dataset(median_X_train, median_X_valid, y_train, y_valid):13.2f}")

MAE from Approach 6 (Imputation with median):
     17791.60


## Results
Of all of these simple strategies `median` performed the best.
Maybe `constant` would perform better if we used the minimum value for the `GarageYrBlt` feature.